In [2]:
from preprocessing import *

In [3]:
from loader_TextLevel import dataset_to_dataframe
import os
import torch
import csv
import numpy as np
import torch
import dgl
import torch.nn.functional as F
import word2vec
import itertools
import torch
import tqdm
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

import sys, random
import argparse
from itertools import compress
import time, datetime
import os
from nltk.tokenize import word_tokenize
import random,re, os, nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from transformers import logging
logging.set_verbosity_error()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
from itertools import compress
import pandas as pd

/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mbugueno/anaconda3/envs/py38_torch111/lib/python3.8/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'


In [4]:
from graph_utils import *
from generation_module import *

## Load data
### App Reviews 


def num_co(a,b,lista, mode='forward'):
    #mode forward co-occurrence from left to right"
    count=0
    for i in range(len(lista)-1):
        if lista[i]==a and lista[i+1]==b:
            count+=1
    return count

def set_nodes(list_text):
    nodes=set(list_text)
    return nodes

def set_edges(list_text, mode='forward', ws=1, weighted=True, directed=True): 
    #mode: 
    #forward, from left to right considering a window size=1 1.C and 1.C2
    #window, given a term A, it defines outgoing edges to the surrounding terms B
    #source -- target (undirected)
    #source -> target (directed)
    edges=[]
    if mode=='forward': #same for directed or not
        for i in range(len(list_text)-1):
            if directed:
                if weighted:
                    times=num_co(list_text[i],list_text[i+1],list_text)
                    if (list_text[i],list_text[i+1], times) not in edges:
                        edges.append((list_text[i],list_text[i+1], times))
                else:
                    edges.append((list_text[i],list_text[i+1]))
            else:
                if weighted:
                    timesa=num_co(list_text[i],list_text[i+1],list_text)
                    timesb=num_co(list_text[i+1],list_text[i],list_text)
                    tuplar=(list_text[i+1],list_text[i],timesa+timesb)
                    if tuplar not in edges:
                        edges.append((list_text[i],list_text[i+1],timesa+timesb))
                        #nuevo
                        edges.append((list_text[i+1],list_text[i],timesa+timesb))
                else:
                    tuplar=(list_text[i+1],list_text[i])
                    if tuplar not in edges:
                        edges.append((list_text[i],list_text[i+1]))
                        # nuevo
                        edges.append((list_text[i+1],list_text[i]))
    
    elif mode=='window':
        dict_edges={}
        #forward
        for i in range(len(list_text)):
            for w in range(1,ws+1):
                if directed:
                    try:
                        if (list_text[i],list_text[i+w]) not in dict_edges.keys():
                            dict_edges[(list_text[i],list_text[i+w])]=1
                    except:
                        continue
                else:
                    try:
                        if (list_text[i],list_text[i+w]) not in dict_edges.keys() and (list_text[i+w],list_text[i]) not in dict_edges.keys():
                            dict_edges[(list_text[i],list_text[i+w])]=1
                        elif (list_text[i],list_text[i+w]) in dict_edges.keys():
                            dict_edges[(list_text[i],list_text[i+w])]+=1
                        elif (list_text[i+w],list_text[i]) in dict_edges.keys():
                            dict_edges[(list_text[i+w],list_text[i])]+=1
                    except:
                        continue
 
        #backward
        list_text.reverse()
        for i in range(len(list_text)):
            for w in range(1,ws+1):
                if directed:
                    try:
                        if (list_text[i],list_text[i+w]) not in dict_edges.keys():
                            dict_edges[(list_text[i],list_text[i+w])]=1
                        else: 
                            dict_edges[(list_text[i],list_text[i+w])]+=1
                    except:
                        continue
                      
        #once the dictionary is created        
        if weighted:                  
            edges=[(tupla[0],tupla[1],dict_edges[tupla]) for tupla in dict_edges.keys()]
            # nuevo
            invertir=[(tupla[1],tupla[0],dict_edges[tupla]) for tupla in dict_edges.keys()]
        else:
            edges=list(dict_edges.keys())
            # nuevo 
            invertir=[(tupla[1],tupla[0]) for tupla in dict_edges.keys()]
        #nuevo    
        edges+=invertir
        
    return edges

def tokenize_sentences(data, labels, tokenizer="nltk"): 
    #obtain sentences 
    #total=len(data)
    #ids_q=[np.random.randint(total) for i in range(num_graphs)]
    labels_return= np.asarray(labels)
    #sents_data = [sent_tokenize(data[ide]) for ide in ids_q]
    #lens_sent = [len(x) for x in sents_data]
    docs=np.asarray(data)
    #print ("\nLenght of sampled text (w.r.t. #sentences):")
    #print ("Min:", np.min(sents), "\tAverage:", np.mean(sents),"\tMax:",np.max(sents))
    #load tokenizer para sentencias
    #tokenized=[]
    
    if tokenizer=="bert":
        mytokenizer = AutoTokenizer.from_pretrained('bert-base-cased')    
        # Parallel(n_jobs=-1)(delayed(clean_text)(sentence) for sentence in sample)
        temp_tokenized = [mytokenizer(sample) for sample in docs]
        tokens=[np.asarray(text['input_ids']) for text in temp_tokenized]
        print ("BERT tokens", tokens)
        #tokens=[np.concatenate(np.asarray(text['input_ids'])) for text in temp_tokenized]
        lens = [len(set(x)) for x in tokens]  ##3lista con ntokens diferentes por review
        all_tokens=np.concatenate([list(set(x)) for x in tokens]) #tenia concatenate 
        for temp in temp_tokenized:
            tokenized.append([mytokenizer.convert_ids_to_tokens(doc[1:-1]) for doc in temp['input_ids']])
        
    else:
        all_tokens=[]
        #for sample in sents_data:
        # Parallel(n_jobs=-1)(delayed(clean_text)(sentence) for sentence in sample)
        temp_tokenized = [word_tokenize(sample) for sample in docs]
        #tokenizer = [wordpunct_tokenize(sample) for sample in data]
        #tokenized.append(temp_tokenized)
        lens = [len(set(sample)) for sample in temp_tokenized]  
        tokens=np.concatenate([list(set(sample)) for sample in temp_tokenized])
        for tokens_review in tokens:
            all_tokens+=set(tokens_review)

    lens=np.asarray(lens)        
    print ("Vocabulary size (",tokenizer,"):", len(set(all_tokens)))
    print ("\nLenght of text (w.r.t. #tokens):")
    print ("Min:", np.min(lens), "\tAverage:", np.mean(lens),"\tMax:",np.max(lens))
    
    return temp_tokenized, labels_return #era tokenized, labels_return

def check(tupla,all_edges): ###entrega True si existe la tupla en edges
    for tup in all_edges:
        if tup[0]==tupla[0] and tup[1]==tupla[1]:
            return True, tup[-1]
    return False, None
    
def update(tupla, cont, all_edges):##re
    all_edges.remove((tupla[0],tupla[1],cont))
    all_edges.append((tupla[0],tupla[1],cont+1))
    return all_edges



In [5]:
def clean_corpus(dataset, df, num=7000, n_short=3, mode="training", preprocess="soft"):

    labels = [str(lab) for lab in df['label'][:num]] 
    if preprocess=="raw":
        corpus = [doc for doc in df["content"][:num]]
    if preprocess!="raw": #preprocessing
        corpus = [clean_str(doc, preprocess) for doc in df["content"][:num]] 
        
    if preprocess=='tlgcn':
        corpus = [remove_short(doc, n_short) for doc in corpus] #n_short  
    
    if mode!="test":
        print ("Processing training set...")
    else:
        print ("Processing testing set...")
    
    
    tokenized_corpus = [word_tokenize(doc) for doc in corpus]

    results = []
    for doc in tokenized_corpus:
        results.append(' '.join([word for word in doc]))
    
    valid_mask=[]
    for doc in results:
        if doc!='':
            valid_mask.append(True)
        else:
            valid_mask.append(False)
    
    final_results=list(compress(results, valid_mask))
    final_labels = list(compress(labels, valid_mask))
    
    print (len(final_results), "/",num, "samples have been generated.")    
    
    results = list(zip(final_labels, final_results))
    
    if mode!="test":
        df = pd.DataFrame(results, columns = ["label", "content"])
        df.to_csv("../Data_RBased/"+dataset+"/"+preprocess+"/source_processed.csv",index=False)
        
    else:    
        df = pd.DataFrame(results, columns = ["label", "content"])
        df.to_csv("../Data_RBased/"+dataset+"/"+preprocess+"/source_processed_test.csv",index=False)
        
        

In [6]:
df_train, df_test = dataset_to_dataframe("app_reviews")

Using custom data configuration default


Dataset app_reviews downloaded and prepared to /home/mbugueno/.cache/huggingface/datasets/app_reviews/default/0.0.0/20335b51b604b9bc04b7be253cd8445caa9ba93f15f39a4b0492b9e9102853de. Subsequent calls will reuse this data.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 310.69it/s]


In [7]:
df_train.head()

,content,label
197527,Wonder ful Fine,4
267684,About my future,4
108964,Well,4
7179,Excellent,4
250988,OK,2


In [11]:
import numpy as np 
np.asarray(df_train['content'])[:10]

array(['Wonder ful Fine', 'About my future', 'Well', 'Excellent', 'OK',
       'Wah',
       "omg. just download it. do it. this app is definitely one of the best things i've come across since using android. it's a must have for all those who want to know what's in their phone  how are apps ruining it's precious free space and want to delete rubbish right away.",
       'Thank is very helpful', 'Way better than the rest',
       'It is very helpful to find apps'], dtype=object)

In [13]:
clean_corpus("app_reviews", df_train, mode="training", num=7000, preprocess="raw")
clean_corpus("app_reviews", df_test, mode="test", num=3000, preprocess="raw")

clean_corpus("app_reviews", df_train, mode="training", num=7000, preprocess="soft")
clean_corpus("app_reviews", df_test, mode="test", num=3000, preprocess="soft")

clean_corpus("app_reviews", df_train, mode="training", num=7000, preprocess="tlgcn")
clean_corpus("app_reviews", df_test, mode="test", num=3000, preprocess="tlgcn")

clean_corpus("app_reviews", df_train, mode="training", num=7000, preprocess="custom")
clean_corpus("app_reviews", df_test, mode="test", num=3000, preprocess="custom")

Processing training set...
7000 / 7000 samples have been generated.
Processing testing set...
3000 / 3000 samples have been generated.
Processing training set...
6971 / 7000 samples have been generated.
Processing testing set...
2992 / 3000 samples have been generated.
Processing training set...
6859 / 7000 samples have been generated.
Processing testing set...
2950 / 3000 samples have been generated.
Processing training set...
7000 / 7000 samples have been generated.
Processing testing set...
3000 / 3000 samples have been generated.


### App Reviews

#### Raw Graphs to Dataframe

* default: raw
* prep: preprocessing text

In [18]:
#1.A 
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_app/1.A/raw/App_graphs_A.csv",index=False)
df.head()

Vocabulary size ( nltk ): 417

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 10.955 	Max: 130


,nodes,node_features,edges,label
0,"[0, 1, 2]","[Wonder, ful, Fine]","[[1, 0, 2, 1], [0, 1, 1, 2]]",4
1,"[0, 1, 2]","[About, future, my]","[[0, 2, 2, 1], [2, 0, 1, 2]]",4
2,[0],[Well],"[[], []]",4
3,[0],[Excellent],"[[], []]",4
4,[0],[OK],"[[], []]",2


In [19]:
#1.A test
df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_app/1.A/raw/App_graphs_A_test.csv",index=False)
df.head()

Vocabulary size ( nltk ): 271

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 11.009 	Max: 116


,nodes,node_features,edges,label
0,"[0, 1, 2, 3]","[play, service, ., Nice]","[[0, 2, 1, 3, 1, 0], [3, 1, 0, 0, 2, 1]]",4
1,"[0, 1]","[Wonderful, Excellence]","[[1, 0], [0, 1]]",4
2,"[0, 1]","[App, Nice]","[[0, 1], [1, 0]]",4
3,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[from, speaking, 👎, f, any, my, much, ., and, ...","[[21, 0, 7, 18, 8, 26, 8, 7, 27, 11, 4, 20, 12...",0
4,"[0, 1, 2, 3]","[Jayesh, 6, bharwad, Saru]","[[3, 2, 1, 0, 2, 3], [2, 0, 3, 2, 3, 1]]",4


In [20]:
#1.A2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_app/1.A2/raw/App_graphs_A2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_app/1.A2/raw/App_graphs_A2_test.csv",index=False)

Vocabulary size ( nltk ): 417

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 10.955 	Max: 130
Vocabulary size ( nltk ): 271

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 11.009 	Max: 116


In [21]:
#1.C
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_app/1.C/raw/App_graphs_C.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_app/1.C/raw/App_graphs_C_test.csv",index=False)

Vocabulary size ( nltk ): 417

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 10.955 	Max: 130
Vocabulary size ( nltk ): 271

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 11.009 	Max: 116


In [22]:
#1.C2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False)
df.to_csv("../Data_Graphs/data_app/1.C2/raw/App_graphs_C2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False)
df.to_csv("../Data_Graphs/data_app/1.C2/raw/App_graphs_C2_test.csv",index=False)

Vocabulary size ( nltk ): 417

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 10.955 	Max: 130
Vocabulary size ( nltk ): 271

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 11.009 	Max: 116


#### Preprocesados

In [23]:
df_train=pd.read_csv("../Data_RBased/app_reviews/soft/source_processed.csv")
df_test=pd.read_csv("../Data_RBased/app_reviews/soft/source_processed_test.csv")
np.asarray(df_train['content'])[0]

'wonder ful fine'

In [24]:
df_train.shape

(6971, 2)

In [25]:
#1.A 
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_app_prep/1.A/raw/App_graphs_A.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_app_prep/1.A/raw/App_graphs_A_test.csv",index=False)

#1.A2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_app_prep/1.A2/raw/App_graphs_A2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_app_prep/1.A2/raw/App_graphs_A2_test.csv",index=False)


Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 6.169559604074021 	Max: 72
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 6.186497326203209 	Max: 67
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 6.169559604074021 	Max: 72
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 6.186497326203209 	Max: 67


In [26]:
#1.C
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_app_prep/1.C/raw/App_graphs_C.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_app_prep/1.C/raw/App_graphs_C_test.csv",index=False)

#1.C2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False)
df.to_csv("../Data_Graphs/data_app_prep/1.C2/raw/App_graphs_C2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False)
df.to_csv("../Data_Graphs/data_app_prep/1.C2/raw/App_graphs_C2_test.csv",index=False)

Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 6.169559604074021 	Max: 72
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 6.186497326203209 	Max: 67
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 6.169559604074021 	Max: 72
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 1 	Average: 6.186497326203209 	Max: 67


In [6]:
from itertools import compress

def clean_all_corpus(datasets):
    
    for dataset in datasets:
        print ("\n--------------------------\nRunning data split for", dataset)
        df_train, df_test = dataset_to_dataframe(dataset)
        
        clean_corpus(dataset, df_train, mode="training", num=7000, preprocess="raw")
        clean_corpus(dataset, df_test, mode="test", num=3000, preprocess="raw")

        clean_corpus(dataset, df_train, mode="training", num=7000, preprocess="soft")
        clean_corpus(dataset, df_test, mode="test", num=3000, preprocess="soft")

        clean_corpus(dataset, df_train, mode="training", num=7000, preprocess="tlgcn")
        clean_corpus(dataset, df_test, mode="test", num=3000, preprocess="tlgcn")

        clean_corpus(dataset, df_train, mode="training", num=7000, preprocess="custom")
        clean_corpus(dataset, df_test, mode="test", num=3000, preprocess="custom")

In [28]:
datasets=["imdb", "dbpedia_14", "hyperpartisan_news_detection", "bbc"]
clean_all_corpus(datasets)


--------------------------
Running data split for imdb


Reusing dataset imdb (/home/mbugueno/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 236.14it/s]


Processing training set...
7000 / 7000 samples have been generated.
Processing testing set...
3000 / 3000 samples have been generated.
Processing training set...
7000 / 7000 samples have been generated.
Processing testing set...
3000 / 3000 samples have been generated.
Processing training set...
7000 / 7000 samples have been generated.
Processing testing set...
3000 / 3000 samples have been generated.
Processing training set...
7000 / 7000 samples have been generated.
Processing testing set...
3000 / 3000 samples have been generated.

--------------------------
Running data split for dbpedia_14


Dataset dbpedia_14 downloaded and prepared to /home/mbugueno/.cache/huggingface/datasets/dbpedia_14/dbpedia_14/2.0.0/01dab9e10d969eadcdbc918be5a09c9190a24caeae33b10eee8f367a1e3f1f0c. Subsequent calls will reuse this data.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 438.51it/s]


...

Extracting data files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 60.52it/s]


Dataset hyperpartisan_news_detection downloaded and prepared to /home/mbugueno/.cache/huggingface/datasets/hyperpartisan_news_detection/byarticle/1.0.0/7f4215b0474950ddf516e806400ab81d098b3da3b3a919a13cd1a4cf2c677012. Subsequent calls will reuse this data.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 558.72it/s]


Processing training set...
516 / 7000 samples have been generated.
Processing testing set...
129 / 3000 samples have been generated.
Processing training set...
516 / 7000 samples have been generated.
Processing testing set...
129 / 3000 samples have been generated.
Processing training set...
516 / 7000 samples have been generated.
Processing testing set...
129 / 3000 samples have been generated.
Processing training set...
516 / 7000 samples have been generated.
Processing testing set...
129 / 3000 samples have been generated.

--------------------------
Running data split for bbc


FileNotFoundError: [Errno 2] No such file or directory: '../Data_Graphs/BBC/Raw.csv'

In [7]:
datasets=["bbc"]
clean_all_corpus(datasets)


--------------------------
Running data split for bbc
Processing training set...
1780 / 7000 samples have been generated.
Processing testing set...
445 / 3000 samples have been generated.
Processing training set...
1780 / 7000 samples have been generated.
Processing testing set...
445 / 3000 samples have been generated.
Processing training set...
1780 / 7000 samples have been generated.
Processing testing set...
445 / 3000 samples have been generated.
Processing training set...
1780 / 7000 samples have been generated.
Processing testing set...
445 / 3000 samples have been generated.


### DBpedia_14

In [8]:
df_train=pd.read_csv("../Data_RBased/dbpedia_14/raw/source_processed.csv")
df_test=pd.read_csv("../Data_RBased/dbpedia_14/raw/source_processed_test.csv")
np.asarray(df_train['content'])[0]

'Naucleopsis oblongifolia is a species of plant in the Moraceae family . It is endemic to Brazil .'

In [9]:
#1.A 
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_db/1.A/raw/DB_graphs_A.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_db/1.A/raw/DB_graphs_A_test.csv",index=False)

#1.A2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_db/1.A2/raw/DB_graphs_A2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_db/1.A2/raw/DB_graphs_A2_test.csv",index=False)

#1.C
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_db/1.C/raw/DB_graphs_C.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_db/1.C/raw/DB_graphs_C_test.csv",index=False)

#1.C2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False)
df.to_csv("../Data_Graphs/data_db/1.C2/raw/DB_graphs_C2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False)
df.to_csv("../Data_Graphs/data_db/1.C2/raw/DB_graphs_C2_test.csv",index=False)

Vocabulary size ( nltk ): 1407

Lenght of text (w.r.t. #tokens):
Min: 3 	Average: 39.409 	Max: 159
Vocabulary size ( nltk ): 995

Lenght of text (w.r.t. #tokens):
Min: 7 	Average: 39.192 	Max: 127
Vocabulary size ( nltk ): 1407

Lenght of text (w.r.t. #tokens):
Min: 3 	Average: 39.409 	Max: 159
Vocabulary size ( nltk ): 995

Lenght of text (w.r.t. #tokens):
Min: 7 	Average: 39.192 	Max: 127
Vocabulary size ( nltk ): 1407

Lenght of text (w.r.t. #tokens):
Min: 3 	Average: 39.409 	Max: 159
Vocabulary size ( nltk ): 995

Lenght of text (w.r.t. #tokens):
Min: 7 	Average: 39.192 	Max: 127
Vocabulary size ( nltk ): 1407

Lenght of text (w.r.t. #tokens):
Min: 3 	Average: 39.409 	Max: 159
Vocabulary size ( nltk ): 995

Lenght of text (w.r.t. #tokens):
Min: 7 	Average: 39.192 	Max: 127


In [10]:
df_train=pd.read_csv("../Data_RBased/dbpedia_14/soft/source_processed.csv")
df_test=pd.read_csv("../Data_RBased/dbpedia_14/soft/source_processed_test.csv")
np.asarray(df_train['content'])[0]

'naucleopsis oblongifolia species plant moraceae family endemic brazil'

In [11]:
#1.A 
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_db_prep/1.A/raw/DB_graphs_A.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_db_prep/1.A/raw/DB_graphs_A_test.csv",index=False)

#1.A2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_db_prep/1.A2/raw/DB_graphs_A2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_db_prep/1.A2/raw/DB_graphs_A2_test.csv",index=False)

#1.C
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_db_prep/1.C/raw/DB_graphs_C.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_db_prep/1.C/raw/DB_graphs_C_test.csv",index=False)

#1.C2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False)
df.to_csv("../Data_Graphs/data_db_prep/1.C2/raw/DB_graphs_C2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False)
df.to_csv("../Data_Graphs/data_db_prep/1.C2/raw/DB_graphs_C2_test.csv",index=False)

Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 2 	Average: 26.325714285714287 	Max: 124
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 3 	Average: 26.165 	Max: 101
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 2 	Average: 26.325714285714287 	Max: 124
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 3 	Average: 26.165 	Max: 101
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 2 	Average: 26.325714285714287 	Max: 124
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 3 	Average: 26.165 	Max: 101
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 2 	Average: 26.325714285714287 	Max: 124
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 3 	Average: 26.165 	Max: 101


### IMDB

In [12]:
df_train=pd.read_csv("../Data_RBased/imdb/raw/source_processed.csv")
df_test=pd.read_csv("../Data_RBased/imdb/raw/source_processed_test.csv")
np.asarray(df_train['content'])[0]

"did n't know anything about the film or that it was based on a best selling book.Tried guessing from the opening scenes what it would be about , best guess , buried treasure and a death list.That lasted about 15 minutes when i got the sneaky suspicion that the film was crap.I 'll not bore you with how bad the plot and acting were but anyone who gave it more than two stars must work for the film makers.I watched until the hero jumped into his corvette to rush to the hospital.He had his on his suit , shirt and tie arrived at the hospital in jeans and a t/shirt.Could n't even get the continuity right . I got the Christian theme , hard to miss it ."

In [13]:
#1.A 
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_imdb/1.A/raw/IMDB_graphs_A.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_imdb/1.A/raw/IMDB_graphs_A_test.csv",index=False)

#1.A2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_imdb/1.A2/raw/IMDB_graphs_A2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_imdb/1.A2/raw/IMDB_graphs_A2_test.csv",index=False)

#1.C
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_imdb/1.C/raw/IMDB_graphs_C.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_imdb/1.C/raw/IMDB_graphs_C_test.csv",index=False)

#1.C2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False)
df.to_csv("../Data_Graphs/data_imdb/1.C2/raw/IMDB_graphs_C2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False)
df.to_csv("../Data_Graphs/data_imdb/1.C2/raw/IMDB_graphs_C2_test.csv",index=False)

Vocabulary size ( nltk ): 146

Lenght of text (w.r.t. #tokens):
Min: 11 	Average: 154.13057142857141 	Max: 692
Vocabulary size ( nltk ): 129

Lenght of text (w.r.t. #tokens):
Min: 8 	Average: 150.429 	Max: 640
Vocabulary size ( nltk ): 146

Lenght of text (w.r.t. #tokens):
Min: 11 	Average: 154.13057142857141 	Max: 692
Vocabulary size ( nltk ): 129

Lenght of text (w.r.t. #tokens):
Min: 8 	Average: 150.429 	Max: 640
Vocabulary size ( nltk ): 146

Lenght of text (w.r.t. #tokens):
Min: 11 	Average: 154.13057142857141 	Max: 692
Vocabulary size ( nltk ): 129

Lenght of text (w.r.t. #tokens):
Min: 8 	Average: 150.429 	Max: 640
Vocabulary size ( nltk ): 146

Lenght of text (w.r.t. #tokens):
Min: 11 	Average: 154.13057142857141 	Max: 692
Vocabulary size ( nltk ): 129

Lenght of text (w.r.t. #tokens):
Min: 8 	Average: 150.429 	Max: 640


In [14]:
df_train=pd.read_csv("../Data_RBased/imdb/soft/source_processed.csv")
df_test=pd.read_csv("../Data_RBased/imdb/soft/source_processed_test.csv")
np.asarray(df_train['content'])[0]

'know anything film based best selling book tried guessing opening scenes would best guess buried treasure death list lasted 15 minutes got sneaky suspicion film crap bore bad plot acting anyone gave two stars must work film makers watched hero jumped corvette rush hospital suit shirt tie arrived hospital jeans shirt even get continuity right got christian theme hard miss'

In [15]:
#1.A 
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_imdb_prep/1.A/raw/IMDB_graphs_A.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_imdb_prep/1.A/raw/IMDB_graphs_A_test.csv",index=False)

#1.A2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_imdb_prep/1.A2/raw/IMDB_graphs_A2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_imdb_prep/1.A2/raw/IMDB_graphs_A2_test.csv",index=False)

#1.C
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_imdb_prep/1.C/raw/IMDB_graphs_C.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_imdb_prep/1.C/raw/IMDB_graphs_C_test.csv",index=False)

#1.C2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False)
df.to_csv("../Data_Graphs/data_imdb_prep/1.C2/raw/IMDB_graphs_C2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False)
df.to_csv("../Data_Graphs/data_imdb_prep/1.C2/raw/IMDB_graphs_C2_test.csv",index=False)

Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 4 	Average: 100.99114285714286 	Max: 551
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 4 	Average: 97.87833333333333 	Max: 518
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 4 	Average: 100.99114285714286 	Max: 551
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 4 	Average: 97.87833333333333 	Max: 518
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 4 	Average: 100.99114285714286 	Max: 551
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 4 	Average: 97.87833333333333 	Max: 518
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 4 	Average: 100.99114285714286 	Max: 551
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 4 	Average: 97.87833333333333 	Max: 518


### hyperpartisan_news_detection

In [16]:
df_train=pd.read_csv("../Data_RBased/hyperpartisan_news_detection/raw/source_processed.csv")
df_test=pd.read_csv("../Data_RBased/hyperpartisan_news_detection/raw/source_processed_test.csv")
np.asarray(df_train['content'])[0]

'< p > CHARLOTTESVILLE , Va. - The reporter of a botched Rolling Stone article about a brutal gang rape at the University of Virginia willfully ignored facts and statements that disproved her preconceived storyline about the school & # 8217 ; s callousness toward sexual-assault victims , an attorney for the former associate dean of students said in his closing arguments Tuesday. < /p > < p > University administrator Nicole Eramo is seeking $ 7.5 million from the magazine over its portrayal of her in the 2014 story by Sabrina Rubin Erdely about the alleged sexual assault of a woman identified only as & # 8220 ; Jackie. & # 8221 ; Eramo claims she was unfairly portrayed in the article as trying to sweep Jackie & # 8217 ; s sexual assault under the rug in order to protect the university. < /p > < p > Attorney Tom Clare argued Tuesday that Erdely set out from the beginning to tell a story of & # 8220 ; institutional indifference , & # 8221 ; brushed off statements from her sources that did

In [17]:
#1.A 
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection/1.A/raw/Hyper_graphs_A.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection/1.A/raw/Hyper_graphs_A_test.csv",index=False)

#1.A2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection/1.A2/raw/Hyper_graphs_A2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection/1.A2/raw/Hyper_graphs_A2_test.csv",index=False)

#1.C
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection/1.C/raw/Hyper_graphs_C.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection/1.C/raw/Hyper_graphs_C_test.csv",index=False)

#1.C2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection/1.C2/raw/Hyper_graphs_C2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection/1.C2/raw/Hyper_graphs_C2_test.csv",index=False)

Vocabulary size ( nltk ): 92

Lenght of text (w.r.t. #tokens):
Min: 21 	Average: 301.8759689922481 	Max: 1402
Vocabulary size ( nltk ): 88

Lenght of text (w.r.t. #tokens):
Min: 55 	Average: 309.0387596899225 	Max: 950
Vocabulary size ( nltk ): 92

Lenght of text (w.r.t. #tokens):
Min: 21 	Average: 301.8759689922481 	Max: 1402
Vocabulary size ( nltk ): 88

Lenght of text (w.r.t. #tokens):
Min: 55 	Average: 309.0387596899225 	Max: 950
Vocabulary size ( nltk ): 92

Lenght of text (w.r.t. #tokens):
Min: 21 	Average: 301.8759689922481 	Max: 1402
Vocabulary size ( nltk ): 88

Lenght of text (w.r.t. #tokens):
Min: 55 	Average: 309.0387596899225 	Max: 950
Vocabulary size ( nltk ): 92

Lenght of text (w.r.t. #tokens):
Min: 21 	Average: 301.8759689922481 	Max: 1402
Vocabulary size ( nltk ): 88

Lenght of text (w.r.t. #tokens):
Min: 55 	Average: 309.0387596899225 	Max: 950


In [18]:
df_train=pd.read_csv("../Data_RBased/hyperpartisan_news_detection/soft/source_processed.csv")
df_test=pd.read_csv("../Data_RBased/hyperpartisan_news_detection/soft/source_processed_test.csv")
np.asarray(df_train['content'])[0]

'p charlottesville va reporter botched rolling stone article brutal gang rape university virginia willfully ignored facts statements disproved preconceived storyline school 8217 callousness toward sexual assault victims attorney former associate dean students said closing arguments tuesday p p university administrator nicole eramo seeking 7 5 million magazine portrayal 2014 story sabrina rubin erdely alleged sexual assault woman identified 8220 jackie 8221 eramo claims unfairly portrayed article trying sweep jackie 8217 sexual assault rug order protect university p p attorney tom clare argued tuesday erdely set beginning tell story 8220 institutional indifference 8221 brushed statements sources 8217 fit narrative pushed views administration onto vulnerable women interviewing p p 8220 decided article going 8217 matter facts 8221 clare said p p story jackie 8217 rape set firestorm university virginia schools nationwide prompted police launch investigation alleged assault eramo received h

In [19]:
#1.A 
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection_prep/1.A/raw/Hyper_graphs_A.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection_prep/1.A/raw/Hyper_graphs_A_test.csv",index=False)

#1.A2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection_prep/1.A2/raw/Hyper_graphs_A2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection_prep/1.A2/raw/Hyper_graphs_A2_test.csv",index=False)

#1.C
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection_prep/1.C/raw/Hyper_graphs_C.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection_prep/1.C/raw/Hyper_graphs_C_test.csv",index=False)

#1.C2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection_prep/1.C2/raw/Hyper_graphs_C2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False)
df.to_csv("../Data_Graphs/data_hyperpartisan_news_detection_prep/1.C2/raw/Hyper_graphs_C2_test.csv",index=False)

Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 8 	Average: 230.51162790697674 	Max: 1240
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 28 	Average: 234.50387596899225 	Max: 824
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 8 	Average: 230.51162790697674 	Max: 1240
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 28 	Average: 234.50387596899225 	Max: 824
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 8 	Average: 230.51162790697674 	Max: 1240
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 28 	Average: 234.50387596899225 	Max: 824
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 8 	Average: 230.51162790697674 	Max: 1240
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 28 	Average: 234.50387596899225 	Max: 824


### BBC

In [20]:
df_train=pd.read_csv("../Data_RBased/bbc/raw/source_processed.csv")
df_test=pd.read_csv("../Data_RBased/bbc/raw/source_processed_test.csv")
np.asarray(df_train['content'])[0]

"Legendary music studio to close The New York music studio where John Lennon spent his final hours is to close next month . The Hit Factory , which opened 37 years ago , has played host to some of the biggest stars in music , including Paul Simon , Madonna and David Bowie . However , the rise in digital recording has affected business at the studio , which is relocating to smaller premises in Miami . Lennon recorded his final album Double Fantasy at the studio in 1979 . The studio was founded by Jerry Ragovoy in 1968 . In 1975 it was sold to fellow producer Edward Germano , who turned it into a 100,000 square foot studio with seven recording rooms and live mastering suites . His first client was Stevie Wonder , who recorded part of his classic album Songs In The Key Of Life there . Other well-known albums to be recorded or partially recorded at the studio include Bruce Springsteen 's Born In The USA , the Rolling Stones ' Emotional Rescue and Paul Simon 's Graceland . Michael Jackson ,

In [21]:
#1.A 
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_bbc/1.A/raw/BBC_graphs_A.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_bbc/1.A/raw/BBC_graphs_A_test.csv",index=False)

#1.A2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_bbc/1.A2/raw/BBC_graphs_A2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_bbc/1.A2/raw/BBC_graphs_A2_test.csv",index=False)

#1.C
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_bbc/1.C/raw/BBC_graphs_C.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_bbc/1.C/raw/BBC_graphs_C_test.csv",index=False)

#1.C2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False)
df.to_csv("../Data_Graphs/data_bbc/1.C2/raw/BBC_graphs_C2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False)
df.to_csv("../Data_Graphs/data_bbc/1.C2/raw/BBC_graphs_C2_test.csv",index=False)

Vocabulary size ( nltk ): 85

Lenght of text (w.r.t. #tokens):
Min: 71 	Average: 215.72359550561796 	Max: 1372
Vocabulary size ( nltk ): 81

Lenght of text (w.r.t. #tokens):
Min: 92 	Average: 225.58876404494382 	Max: 1020
Vocabulary size ( nltk ): 85

Lenght of text (w.r.t. #tokens):
Min: 71 	Average: 215.72359550561796 	Max: 1372
Vocabulary size ( nltk ): 81

Lenght of text (w.r.t. #tokens):
Min: 92 	Average: 225.58876404494382 	Max: 1020
Vocabulary size ( nltk ): 85

Lenght of text (w.r.t. #tokens):
Min: 71 	Average: 215.72359550561796 	Max: 1372
Vocabulary size ( nltk ): 81

Lenght of text (w.r.t. #tokens):
Min: 92 	Average: 225.58876404494382 	Max: 1020
Vocabulary size ( nltk ): 85

Lenght of text (w.r.t. #tokens):
Min: 71 	Average: 215.72359550561796 	Max: 1372
Vocabulary size ( nltk ): 81

Lenght of text (w.r.t. #tokens):
Min: 92 	Average: 225.58876404494382 	Max: 1020


In [22]:
df_train=pd.read_csv("../Data_RBased/bbc/soft/source_processed.csv")
df_test=pd.read_csv("../Data_RBased/bbc/soft/source_processed_test.csv")
np.asarray(df_train['content'])[0]

'legendary music studio close new york music studio john lennon spent final hours close next month hit factory opened 37 years ago played host biggest stars music including paul simon madonna david bowie however rise digital recording affected business studio relocating smaller premises miami lennon recorded final album double fantasy studio 1979 studio founded jerry ragovoy 1968 1975 sold fellow producer edward germano turned 100 000 square foot studio seven recording rooms live mastering suites first client stevie wonder recorded part classic album songs key life well known albums recorded partially recorded studio include bruce springsteen born usa rolling stones emotional rescue paul simon graceland michael jackson billy joel jay z beyonce also among artists used hit factory past well 50 cent survived attempt life left premises 2000 studio made history 1994 record 41 grammy nominations songs recorded mastered mixed including soundtrack whitney houston film bodyguard'

In [24]:
#1.A 
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_bbc_prep/1.A/raw/BBC_graphs_A.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='forward', ws=1, directed=False)
df.to_csv("../Data_Graphs/data_bbc_prep/1.A/raw/BBC_graphs_A_test.csv",index=False)

#1.A2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_bbc_prep/1.A2/raw/BBC_graphs_A2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=False, mode='window', ws=2, directed=False)
df.to_csv("../Data_Graphs/data_bbc_prep/1.A2/raw/BBC_graphs_A2_test.csv",index=False)

#1.C
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_bbc_prep/1.C/raw/BBC_graphs_C.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False, weighted=True, mode='forward', ws=1, directed=True)
df.to_csv("../Data_Graphs/data_bbc_prep/1.C/raw/BBC_graphs_C_test.csv",index=False)

#1.C2
df=create_graph2Dataframe(df_train['content'], df_train['label'], plot=False)
df.to_csv("../Data_Graphs/data_bbc_prep/1.C2/raw/BBC_graphs_C2.csv",index=False)

df=create_graph2Dataframe(df_test['content'], df_test['label'], plot=False)
df.to_csv("../Data_Graphs/data_bbc_prep/1.C2/raw/BBC_graphs_C2_test.csv",index=False)

Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 42 	Average: 158.19044943820225 	Max: 1149
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 64 	Average: 165.59101123595505 	Max: 819
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 42 	Average: 158.19044943820225 	Max: 1149
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 64 	Average: 165.59101123595505 	Max: 819
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 42 	Average: 158.19044943820225 	Max: 1149
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 64 	Average: 165.59101123595505 	Max: 819
Vocabulary size ( nltk ): 37

Lenght of text (w.r.t. #tokens):
Min: 42 	Average: 158.19044943820225 	Max: 1149
Vocabulary size ( nltk ): 36

Lenght of text (w.r.t. #tokens):
Min: 64 	Average: 165.59101123595505 	Max: 819
